In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)


learning_rate = .01
iteration = 50
batch_size = 1000
num_batches = mnist.train.num_examples // batch_size
display_steps = 10


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
input_size = 784
hidden1_size = 254
hidden2_size = 128

In [4]:
weight_variables = {
    'encoder_h1':tf.Variable(tf.random_normal([input_size,hidden1_size])),
    'encoder_h2':tf.Variable(tf.random_normal([hidden1_size,hidden2_size])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden2_size,hidden1_size])),
    'decoder_h2':tf.Variable(tf.random_normal([hidden1_size,input_size]))
}

biases = {
    'encoder_h1':tf.Variable(tf.random_normal([hidden1_size])),
    'encoder_h2':tf.Variable(tf.random_normal([hidden2_size])),
    'decoder_h1':tf.Variable(tf.random_normal([hidden1_size])),
    'decoder_h2':tf.Variable(tf.random_normal([input_size]))
}

In [5]:
def encoder(input,weights,bias):
    layer1 = tf.nn.sigmoid(tf.add(
                            tf.matmul(input,weights['encoder_h1']),
                            bias['encoder_h1']))
    layer2 = tf.nn.sigmoid(tf.add(
                            tf.matmul(layer1,weights['encoder_h2']),
                            bias['encoder_h2']))
    return layer2


def decoder(input,weights,bias):
    layer1 = tf.nn.sigmoid(tf.add(
                            tf.matmul(input,weights['decoder_h1']),
                            bias['decoder_h1']))
    layer2 = tf.nn.sigmoid(tf.add(
                            tf.matmul(layer1,weights['decoder_h2']),
                            bias['decoder_h2']))
    return layer2



In [6]:
X = tf.placeholder(tf.float32,[None,784])

encoded = encoder(X,weight_variables,biases)
decoded = decoder(encoded,weight_variables,biases)

y_pred = decoded
y_true = X

cost = tf.reduce_mean(tf.square(y_pred-y_true))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()




In [7]:
sess = tf.Session()
sess.run(init)
# with tf.Session() as sess:
#     sess.run(init)
    
for i in range(10):
        
        for b in range(num_batches):
                batch_x,_ = mnist.train.next_batch(batch_size)
                _,loss = sess.run([optimizer,cost],feed_dict={X:batch_x})
        
#         if i % display_steps or i == 1 :
        print("Step {} have loss {}".format(i,loss))
                
                

Step 0 have loss 0.4298338294029236
Step 1 have loss 0.4109456241130829
Step 2 have loss 0.21990448236465454
Step 3 have loss 0.17330676317214966
Step 4 have loss 0.1673019975423813
Step 5 have loss 0.15715466439723969
Step 6 have loss 0.1513802856206894
Step 7 have loss 0.14700938761234283
Step 8 have loss 0.13998748362064362
Step 9 have loss 0.13530616462230682


In [ ]:
n = 4

predicted_region =  np.empty([n*28,n*28])
original_region = np.empty([n*28,n*28])
for i in range(n):
    
    batch_x , _ = mnist.train.next_batch(n)
    
    predict = sess.run(decoded,feed_dict={X:batch_x})
    
    for j in range(n):
     
        predicted_region[i*28:(i+1)*28,j*28:(j+1)*28] = predict[j].reshape([28,28])
        
        original_region[i*28:(i+1)*28,j*28:(j+1)*28] = batch_x[j].reshape([28,28])
        
import matplotlib.pyplot as plt

print("Original image")
plt.imshow(original_region)
plt.show()

print("\n\n Predicted Region")
plt.imshow(predicted_region)
plt.show()